# Ames House price series

In [1]:
# Load tools
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import matplotlib as mpl
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('KaggleAmes.csv')

In [3]:
# drop a bunch of useless series
bigDrop = False
if (bigDrop):
    df.drop(df.columns[[0,2,6, 7, 8,9,10,11,12,13,14,21,22,23,24,25,27,28,29,
                        32,33,35,36,39,40,42,45,47,48,49,50,52,53,55,57,60,63,
                        64,65,68,69,70,71,72,73,74,75,76,78,79]],axis=1,inplace=True)
if(not(bigDrop)):
    df.drop(df.columns[[0]],axis=1,inplace=True)

In [4]:
print(df.describe())
print(df.shape)

        MSSubClass  LotFrontage        LotArea  OverallQual  OverallCond  \
count  1460.000000  1201.000000    1460.000000  1460.000000  1460.000000   
mean     56.897260    70.049958   10516.828082     6.099315     5.575342   
std      42.300571    24.284752    9981.264932     1.382997     1.112799   
min      20.000000    21.000000    1300.000000     1.000000     1.000000   
25%      20.000000    59.000000    7553.500000     5.000000     5.000000   
50%      50.000000    69.000000    9478.500000     6.000000     5.000000   
75%      70.000000    80.000000   11601.500000     7.000000     6.000000   
max     190.000000   313.000000  215245.000000    10.000000     9.000000   

         YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1   BsmtFinSF2  ...  \
count  1460.000000   1460.000000  1452.000000  1460.000000  1460.000000  ...   
mean   1971.267808   1984.865753   103.685262   443.639726    46.549315  ...   
std      30.202904     20.645407   181.066207   456.098091   161.319273  ..

In [5]:
# display all variable names
print(df.columns)
df["SaleType"]

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

0       WD
1       WD
2       WD
3       WD
4       WD
        ..
1455    WD
1456    WD
1457    WD
1458    WD
1459    WD
Name: SaleType, Length: 1460, dtype: object

In [6]:
# drop series with similar stuff
# Shift param down to 0.95
sameThresh = 1.00
dropSameList = []
for i in range(df.shape[1]):
    xsame = np.mean( df.iloc[0,i]==df.iloc[:,i])
    if(xsame>sameThresh):
        dropSameList.append(i) 
df.drop(df.columns[[dropSameList]],axis=1,inplace=True)

In [7]:
# Check NA's in each column
# drop variables with > missingThresh
missingThresh = 50.
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
drop_col = missing_value_df[missing_value_df.percent_missing > missingThresh]
missing_list = drop_col.column_name.values
df.drop(missing_list,axis=1,inplace=True)

In [8]:
# generate dummies
df = pd.get_dummies(df)
# fill na's with means
df = df.fillna(df.mean())
print(df.describe())

        MSSubClass  LotFrontage        LotArea  OverallQual  OverallCond  \
count  1460.000000  1460.000000    1460.000000  1460.000000  1460.000000   
mean     56.897260    70.049958   10516.828082     6.099315     5.575342   
std      42.300571    22.024023    9981.264932     1.382997     1.112799   
min      20.000000    21.000000    1300.000000     1.000000     1.000000   
25%      20.000000    60.000000    7553.500000     5.000000     5.000000   
50%      50.000000    70.049958    9478.500000     6.000000     5.000000   
75%      70.000000    79.000000   11601.500000     7.000000     6.000000   
max     190.000000   313.000000  215245.000000    10.000000     9.000000   

         YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1   BsmtFinSF2  ...  \
count  1460.000000   1460.000000  1460.000000  1460.000000  1460.000000  ...   
mean   1971.267808   1984.865753   103.685262   443.639726    46.549315  ...   
std      30.202904     20.645407   180.569112   456.098091   161.319273  ..

In [9]:
#change y to SalesPrice log 
# y = np.log(df.SalePrice)
y = df.SalePrice

#drop original dependent variable
X = df.drop(['SalePrice'], axis=1)

In [10]:
 # Linear regression
nmc = 100
lr = LinearRegression()
TestRsquareVec = np.zeros(nmc)
TrainRsquareVec = np.zeros(nmc)
for i in range(nmc):
    # set train/test samples
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5)
    # Run linear regression
    lr.fit(X_train, y_train)
    # print scores, which is R-squared (fit)
    TestRsquareVec[i] = lr.score(X_test,y_test)
    TrainRsquareVec[i] = lr.score(X_train,y_train)
    
# Statistics
print("Train R-squared mean: {:.4f}".format(np.mean(TrainRsquareVec)))
print("Train R-squared standard deviation: {:.4f}".format(np.std(TrainRsquareVec)))
print("Test R-squared mean: {:.4f}".format(np.mean(TestRsquareVec)))
print("Test R-squared standard deviation: {:.4f}".format(np.std(TestRsquareVec)))

Train R-squared mean: 0.9482
Train R-squared standard deviation: 0.0080
Test R-squared mean: -86653293.6125
Test R-squared standard deviation: 708471362.6320


In [17]:
# Ridge Regression
from sklearn.linear_model import Ridge
alpha_settings = np.array([0.5, 1.0, 2.0, 3.0]) #try bigger alphas
nmc = 50

for a in alpha_settings:
    ridge = Ridge(alpha=a)
    trainScore = np.zeros(nmc)
    testScore  = np.zeros(nmc)
    for n in range(nmc):
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
        scaler = StandardScaler().fit(X_train)
        X_train2 = scaler.transform(X_train)
        X_test2  = scaler.transform(X_test)
        ridge.fit(X_train2, y_train)
        testScore[n] = ridge.score(X_test2,y_test)
        trainScore[n] = ridge.score(X_train2,y_train)
    print('alpha = '+str(a)+":")
    print("Train R-squared mean: {:.4f}".format(np.mean(trainScore)))
    print("Train R-squared standard deviation: {:.4f}".format(np.std(trainScore)))
    print("Test R-squared mean: {:.4f}".format(np.mean(testScore)))
    print("Test R-squared standard deviation: {:.4f}".format(np.std(testScore)))
    print(end="\n")

alpha = 0.5:
Train R-squared mean: 0.9384
Train R-squared standard deviation: 0.0057
Test R-squared mean: 0.8124
Test R-squared standard deviation: 0.0935

alpha = 1.0:
Train R-squared mean: 0.9384
Train R-squared standard deviation: 0.0062
Test R-squared mean: 0.7991
Test R-squared standard deviation: 0.0945

alpha = 2.0:
Train R-squared mean: 0.9404
Train R-squared standard deviation: 0.0052
Test R-squared mean: 0.7981
Test R-squared standard deviation: 0.1000

alpha = 3.0:
Train R-squared mean: 0.9378
Train R-squared standard deviation: 0.0055
Test R-squared mean: 0.7778
Test R-squared standard deviation: 0.1227



In [12]:
# Decision Trees
from sklearn.tree import DecisionTreeRegressor
nmc = 50
depths = [5, 10, 15]
for d in depths:
    tree = DecisionTreeRegressor(max_depth=d)
    trainScore = np.zeros(nmc)
    testScore  = np.zeros(nmc)
    for i in range(nmc):
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
        trainFit = tree.fit(X_train,y_train)
        trainScore[i] = trainFit.score(X_train,y_train)
        testScore[i] =  trainFit.score(X_test,y_test)
    print('max_depth = '+str(d)+":")
    print("Train R-squared mean: {:.4f}".format(np.mean(trainScore)))
    print("Train R-squared standard deviation: {:.4f}".format(np.std(trainScore)))
    print("Test R-squared mean: {:.4f}".format(np.mean(testScore)))
    print("Test R-squared standard deviation: {:.4f}".format(np.std(testScore)))
    print(end="\n")

max_depth = 5:
Train R-squared mean: 0.8705
Train R-squared standard deviation: 0.0085
Test R-squared mean: 0.6976
Test R-squared standard deviation: 0.0787

max_depth = 10:
Train R-squared mean: 0.9876
Train R-squared standard deviation: 0.0022
Test R-squared mean: 0.7238
Test R-squared standard deviation: 0.0756

max_depth = 15:
Train R-squared mean: 0.9994
Train R-squared standard deviation: 0.0003
Test R-squared mean: 0.7185
Test R-squared standard deviation: 0.0794



In [13]:
# random forest
from sklearn.ensemble import RandomForestRegressor    
doForest = True
if (doForest)  :
    nmc = 50
    depths = [5, 10, 15]
    ntree = 100
    for d in depths:
        print('max_depth = '+str(d)+":")
        
        features = [5, 10, 25, 50]
        for f in features:
            forest = RandomForestRegressor(max_depth=d, n_estimators=ntree, max_features=f)
            trainScore = np.zeros(nmc)
            testScore  = np.zeros(nmc)
        
            for i in range(nmc):
                X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
                trainFit = forest.fit(X_train,y_train)
                trainScore[i] = trainFit.score(X_train,y_train)
                testScore[i] =  trainFit.score(X_test,y_test)
                
            print('    max_features = '+str(f)+":")
            print('    Train R-squared mean: {:.4f}'.format(np.mean(trainScore)))
            # print("Train R-squared standard deviation: {:.4f}".format(np.std(trainScore)))
            print('    Test R-squared mean: {:.4f}'.format(np.mean(testScore)))
            # print("Test R-squared standard deviation: {:.4f}".format(np.std(testScore)))

max_depth = 5:
    max_features = 5:
    Train R-squared mean: 0.7239
    Test R-squared mean: 0.6711
    max_features = 10:
    Train R-squared mean: 0.8183
    Test R-squared mean: 0.7462
    max_features = 25:
    Train R-squared mean: 0.8799
    Test R-squared mean: 0.7969
    max_features = 50:
    Train R-squared mean: 0.9030
    Test R-squared mean: 0.8132
max_depth = 10:
    max_features = 5:
    Train R-squared mean: 0.9136
    Test R-squared mean: 0.7782
    max_features = 10:
    Train R-squared mean: 0.9497
    Test R-squared mean: 0.8243
    max_features = 25:
    Train R-squared mean: 0.9677
    Test R-squared mean: 0.8525
    max_features = 50:
    Train R-squared mean: 0.9729
    Test R-squared mean: 0.8572
max_depth = 15:
    max_features = 5:
    Train R-squared mean: 0.9626
    Test R-squared mean: 0.8050
    max_features = 10:
    Train R-squared mean: 0.9732
    Test R-squared mean: 0.8311
    max_features = 25:
    Train R-squared mean: 0.9781
    Test R-squared m